In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.parser import parse
from dotenv import dotenv_values
from pybit.unified_trading import HTTP

In [2]:
def open_long():
    while True:
        try:
            balance = float(session.get_wallet_balance(accountType="UNIFIED")['result']['list'][0]['coin'][0]['walletBalance'])
            price = float(session.get_tickers(category="linear",symbol="BTCUSDT")['result']['list'][0]['lastPrice'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = pos_pct * balance / price * leverage
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Buy", orderType="Market", qty=size,
                                stopLoss="{:.2f}".format(((1 - sl / leverage) * price)))
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [3]:
def open_short():
    while True:
        try:
            balance = float(session.get_wallet_balance(accountType="UNIFIED")['result']['list'][0]['coin'][0]['walletBalance'])
            price = float(session.get_tickers(category="linear",symbol="BTCUSDT")['result']['list'][0]['lastPrice'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = pos_pct * balance / price * leverage
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Sell", orderType="Market", qty=size,
                                stopLoss="{:.2f}".format(((1 + sl / leverage) * price)))
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [4]:
def close_long():
    while True:
        try:
            size = float(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['size'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Sell", orderType="Market", qty=str(size),
                                reduceOnly=True)
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [5]:
def close_short():
    while True:
        try:
            size = float(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['size'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    size = "{:.3f}".format(size)
    while True:
        try:
            session.place_order(category="linear", symbol="BTCUSDT", side="Buy", orderType="Market", qty=str(size),
                                reduceOnly=True)
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)

In [2]:
config = dotenv_values('.env')
API_KEY = config.get('BYBIT_API_KEY')
API_SECRET = config.get('BYBIT_API_SECRET')

In [3]:
session = HTTP(testnet=False, api_key=API_KEY, api_secret=API_SECRET)

In [4]:
# Params

tf = 60
w = 14
std_multiple = 3

sl = 0.1
leverage = 10
pos_pct = 0.9

In [5]:
assert int(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['leverage']) == leverage

In [85]:
data = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=24)['result']['list']

In [86]:
[d[0] for d in data]

['1693162800000',
 '1693159200000',
 '1693155600000',
 '1693152000000',
 '1693148400000',
 '1693144800000',
 '1693141200000',
 '1693137600000',
 '1693134000000',
 '1693130400000',
 '1693126800000',
 '1693123200000',
 '1693119600000',
 '1693116000000',
 '1693112400000',
 '1693108800000',
 '1693105200000',
 '1693101600000',
 '1693098000000',
 '1693094400000',
 '1693090800000',
 '1693087200000',
 '1693083600000',
 '1693080000000']

In [87]:
np.array([[d[0] for d in data], [d[1] for d in data], [d[2] for d in data], [d[3] for d in data],
                                 [d[4] for d in data]]).shape

(5, 24)

In [88]:
df = pd.DataFrame(data=np.array([[d[0] for d in data], [d[1] for d in data], [d[2] for d in data], [d[3] for d in data],
                                 [d[4] for d in data]]).T, columns=['time', 'open', 'high', 'low', 'close']).astype(float)

In [89]:
df['time'] = (df['time'] / 1000).apply(datetime.fromtimestamp)

In [90]:
df = df.set_index('time', drop=True)

In [91]:
df = df.sort_index()

In [92]:
df['size'] = df['high'] - df['low']

In [93]:
df['mean_size'] = df['size'].rolling(14).mean()

In [94]:
df['std_size'] = df['size'].rolling(14).std()

In [95]:
df['ratio'] = (df['size'] - df['mean_size']) / df['std_size']

In [96]:
df

,open,high,low,close,size,mean_size,std_size,ratio
time,,,,,,,,
2023-08-26 22:00:00,26023.3,26037.3,26013.5,26021.1,23.8,NaN,NaN,NaN
2023-08-26 23:00:00,26021.1,26038.2,26001.0,26032.3,37.2,NaN,NaN,NaN
2023-08-27 00:00:00,26032.3,26034.2,26020.7,26020.7,13.5,NaN,NaN,NaN
2023-08-27 01:00:00,26020.7,26020.8,25975.1,26008.5,45.7,NaN,NaN,NaN
2023-08-27 02:00:00,26008.5,26016.9,25987.4,25987.5,29.5,NaN,NaN,NaN
2023-08-27 03:00:00,25987.5,25998.9,25957.3,25983.9,41.6,NaN,NaN,NaN
2023-08-27 04:00:00,25983.9,26001.0,25977.9,25983.4,23.1,NaN,NaN,NaN
2023-08-27 05:00:00,25983.4,26009.8,25983.4,26009.7,26.4,NaN,NaN,NaN
2023-08-27 06:00:00,26009.7,26021.0,26008.1,26019.9,12.9,NaN,NaN,NaN


In [83]:
candles = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=w + 1)['result']['list']

In [97]:
last_w_candles = candles[1:]
curr_candle = candles[0]

In [98]:
last_w_candle_sizes = [float(x[2]) - float(x[3]) for x in last_w_candles]
last_w_candle_sizes_mean = np.mean(last_w_candle_sizes)
last_w_candle_sizes_std = np.std(last_w_candle_sizes, ddof=1)
thr = last_w_candle_sizes_mean + std_multiple * last_w_candle_sizes_std

In [100]:
last_w_candle_sizes_mean, last_w_candle_sizes_std, thr

(43.09285714285683, 38.27205174869682, 157.9090123889473)

# Trading

In [ ]:
first_candle = True

while True:
    try:
        candles = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=w + 1)['result']['list']
        break
    except ConnectionError:
        print('Connection error. Trying again...')
        time.sleep(1)
last_w_candles = candles[1:]
curr_candle = candles[0]

last_w_candle_sizes = [float(x[2]) - float(x[3]) for x in last_w_candles]
last_w_candle_sizes_mean = np.mean(last_w_candle_sizes)
last_w_candle_sizes_std = np.std(last_w_candle_sizes, ddof=1)
thr = last_w_candle_sizes_mean + std_multiple * last_w_candle_sizes_std

curr_ts = int(curr_candle[0])
next_ts = curr_ts + 3600000

while True:
    while True:
        try:
            pos_size = float(session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['size'])
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    if 1000 * time.time() >= next_ts:
        if pos_size > 0:
            print('Closing position...')
            while True:
                try:
                    side = session.get_positions(category="linear", symbol="BTCUSDT")['result']['list'][0]['side']
                    break
                except ConnectionError:
                    print('Connection error. Trying again...')
                    time.sleep(1)
            if side == 'Buy':
                close_long()
            elif side == 'Sell':
                close_short()
        print('Next candle...\n')
        first_candle = False
        
        while True:
            try:
                candles = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=w + 1)['result']['list']
                break
            except ConnectionError:
                print('Connection error. Trying again...')
                time.sleep(1)
        last_w_candles = candles[1:]
        curr_candle = candles[0]
        last_w_candle_sizes = [float(x[2]) - float(x[3]) for x in last_w_candles]
        last_w_candle_sizes_mean = np.mean(last_w_candle_sizes)
        last_w_candle_sizes_std = np.std(last_w_candle_sizes, ddof=1)
        thr = last_w_candle_sizes_mean + std_multiple * last_w_candle_sizes_std
        curr_ts = int(curr_candle[0])
        next_ts = curr_ts + 3600000
    
    while True:
        try:
            curr_candle = session.get_kline(category="linear", symbol="BTCUSDT", interval=tf, limit=1)['result']['list'][0]
            break
        except ConnectionError:
            print('Connection error. Trying again...')
            time.sleep(1)
    curr_candle_size = float(curr_candle[2]) - float(curr_candle[3])
    
    if not first_candle and (pos_size == 0) and (curr_candle_size >= thr):
        print(f'Current candle size ({curr_candle_size}) > threshold ({thr})')
        if float(curr_candle[4]) > float(curr_candle[1]):  # if close > open
            print('Opening short...')
            open_short()
        elif float(curr_candle[4]) < float(curr_candle[1]):  # if close < open
            print('Opening long...')
            open_long()
    
    time.sleep(1)